# Text Summarization using


## 1. Setup & Installation

In [1]:
!pip install -q transformers datasets torch pandas accelerate sentencepiece

In [2]:
# Import libraries
import torch
from transformers import pipeline
from datasets import load_dataset
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

print("✅ Setup complete!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

✅ Setup complete!
PyTorch: 2.9.0+cu126
CUDA Available: True
Device: GPU


## 2. Model Selection & Justification

### Primary Model: facebook/bart-large-cnn

**BART-CNN is the industry standard for news summarization.

In [3]:
# Model configurations
MODELS = {
    "BART-CNN": {
        "name": "facebook/bart-large-cnn",
        "max_len": 142,
        "min_len": 56,
        "description": "Production standard - informative 3-4 sentence summaries"
    },
    "PEGASUS-XSum": {
        "name": "google/pegasus-xsum",
        "max_len": 64,
        "min_len": 10,
        "description": "Academic - single sentence extreme summarization"
    },
    "LED-16K": {
        "name": "allenai/led-base-16384",
        "max_len": 142,
        "min_len": 56,
        "description": "Specialized - handles very long documents"
    }
}

print("📊 Available Models:")
for name, config in MODELS.items():
    print(f"\n{name}:")
    print(f"  Model: {config['name']}")
    print(f"  Description: {config['description']}")

📊 Available Models:

BART-CNN:
  Model: facebook/bart-large-cnn
  Description: Production standard - informative 3-4 sentence summaries

PEGASUS-XSum:
  Model: google/pegasus-xsum
  Description: Academic - single sentence extreme summarization

LED-16K:
  Model: allenai/led-base-16384
  Description: Specialized - handles very long documents


## 3. Data Loading & Preprocessing

### Library Choices:

- **datasets (HuggingFace)**: Native integration, efficient loading
- **pandas**: Data manipulation and analysis
- **torch**: GPU acceleration, tensor operations

In [4]:
# Load XSum dataset
print("📥 Loading XSum dataset...")
dataset = load_dataset("xsum", split="test")
samples = dataset.select(range(50))  # 50 samples for demo

print(f"✅ Loaded {len(samples)} samples")
print(f"\nDataset structure: {samples.column_names}")
print(f"First example keys: {samples[0].keys()}")

📥 Loading XSum dataset...


README.md: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 05a84d67-5dbb-4ecd-af47-5651548517a7)')' thrown while requesting HEAD https://huggingface.co/datasets/xsum/resolve/7d4d486c2f8ef850b1a11aead99b894ff3dd7da9/data/train-00000-of-00001.parquet
Retrying in 1s [Retry 1/5].


data/train-00000-of-00001.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

...

data/test-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

✅ Loaded 50 samples

Dataset structure: ['document', 'summary', 'id']
First example keys: dict_keys(['document', 'summary', 'id'])


In [5]:
# Inspect sample data
print("📊 Sample Data Inspection:\n")
print("=" * 80)

for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"Article (first 200 chars): {samples[i]['document'][:200]}...")
    print(f"XSum Reference: {samples[i]['summary']}")
    print("-" * 80)

📊 Sample Data Inspection:


Example 1:
Article (first 200 chars): Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing...
XSum Reference: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
--------------------------------------------------------------------------------

Example 2:
Article (first 200 chars): Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday.
Detectives said three firearms, ammunition and a five-figure sum of money were recovered.
A 26-yea...
XSum Reference: A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh.
--------------------------------------------------------------------------------

Example 3:
Article (first 200 chars): Jordan Hill, Brittany Covington and Tesfaye Cooper, all 18

## 4. Pipeline Implementation

### Pipeline Benefits:

```
Input Text → Tokenization → Model → Decoding → Summary
```

The `pipeline()` abstraction handles:
- ✅ Automatic tokenization
- ✅ Model inference
- ✅ Decoding
- ✅ Batch processing
- ✅ GPU acceleration

In [6]:
# Initialize BART-CNN (Primary Model)
print("🚀 Loading BART-CNN model...")
print("⚠️  First run downloads ~2GB - subsequent runs are fast\n")

device = 0 if torch.cuda.is_available() else -1

bart_summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=device
)

print("✅ BART-CNN ready!")
print(f"   Device: {'GPU' if device == 0 else 'CPU'}")

🚀 Loading BART-CNN model...
⚠️  First run downloads ~2GB - subsequent runs are fast



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


✅ BART-CNN ready!
   Device: GPU


## 5. Single Text Summarization

In [7]:
# Test with first example
test_text = samples[0]['document']
xsum_ref = samples[0]['summary']

print("📄 Input Article (first 400 chars):")
print("=" * 80)
print(test_text[:400] + "...\n")

# Generate summary
print("⏳ Generating summary...")
start = time.time()

# Added truncation=True to prevent index out of bounds errors
result = bart_summarizer(
    test_text,
    max_length=142,
    min_length=56,
    do_sample=False,
    truncation=True
)

elapsed = time.time() - start
summary = result[0]['summary_text']

print("\n" + "=" * 80)
print("📝 BART-CNN Summary (3-4 sentences):")
print("=" * 80)
print(summary)

print("\n" + "=" * 80)
print("🎯 XSum Reference (1 sentence):")
print("=" * 80)
print(xsum_ref)

print("\n" + "=" * 80)
print("📊 Metrics:")
print("=" * 80)
print(f"Input length: {len(test_text.split())} words")
print(f"BART summary: {len(summary.split())} words")
print(f"XSum reference: {len(xsum_ref.split())} words")
print(f"Inference time: {elapsed:.2f} seconds")
print(f"Compression ratio: {len(summary.split())/len(test_text.split())*100:.1f}%")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


📄 Input Article (first 400 chars):
Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced...

⏳ Generating summary...

📝 BART-CNN Summary (3-4 sentences):
Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation. Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders. Welsh Government said more people than ever were getting help to address housing problems.

🎯 XSum Reference (1 sentence):
There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.

📊 Metrics:
Input leng

## 6. Model Comparison (BART vs PEGASUS vs LED)

This demonstrates understanding of different model characteristics.

In [8]:
# Prepare for sequential comparison (Save Memory)
import gc
import torch

print("🔄 Preparing for model comparison...")
print("To save GPU memory, we will load models sequentially in the next cell rather than all at once.")

# Clear GPU memory from previous cells to avoid OOM
if 'bart_summarizer' in globals():
    print("Unloading initial BART model to free space...")
    del bart_summarizer

if 'models_loaded' in globals():
    del models_loaded

try:
    torch.cuda.empty_cache()
    gc.collect()
    print("✅ Memory cleared. Ready for sequential comparison.")
except Exception as e:
    print(f"⚠️ Memory clear failed: {e}")
    print("❗ CRITICAL: The GPU is in a bad state (Device-side assert). You MUST restart the runtime (Runtime > Restart session) to continue.")

🔄 Preparing for model comparison...
To save GPU memory, we will load models sequentially in the next cell rather than all at once.
Unloading initial BART model to free space...
✅ Memory cleared. Ready for sequential comparison.


In [9]:
# Compare on same text using Sequential Loading
comparison_results = []

print("⚖️  Comparing Models on Same Article (Sequential Load/Unload)\n")
print("=" * 80)

for model_name, config in MODELS.items():
    print(f"\n{model_name}:")
    print("-" * 80)

    try:
        print(f"Loading {model_name}...")
        # Load model just-in-time
        current_model = pipeline(
            "summarization",
            model=config['name'],
            device=device
        )

        start = time.time()
        # Added truncation=True to prevent CUDA asserts on long docs
        result = current_model(
            test_text,
            max_length=config['max_len'],
            min_length=config['min_len'],
            do_sample=False,
            truncation=True
        )
        elapsed = time.time() - start

        summary = result[0]['summary_text']

        print(f"Summary: {summary}")
        print(f"Length: {len(summary.split())} words")
        print(f"Time: {elapsed:.2f}s")

        comparison_results.append({
            'Model': model_name,
            'Summary': summary,
            'Words': len(summary.split()),
            'Time (s)': f"{elapsed:.2f}",
            'Description': config['description']
        })

        # IMMEDIATE CLEANUP to save memory
        del current_model
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"❌ Error during summarization with {model_name}: {e}")
        comparison_results.append({
            'Model': model_name,
            'Summary': 'Error: Could not generate summary',
            'Words': 0,
            'Time (s)': 'N/A',
            'Description': config['description']
        })

print("\n" + "=" * 80)

⚖️  Comparing Models on Same Article (Sequential Load/Unload)


BART-CNN:
--------------------------------------------------------------------------------
Loading BART-CNN...


Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Summary: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation. Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders. Welsh Government said more people than ever were getting help to address housing problems.
Length: 54 words
Time: 1.27s

PEGASUS-XSum:
--------------------------------------------------------------------------------
Loading PEGASUS-XSum...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0
Input ids are automatically padded from 748 to 1024 to be a multiple of `config.attention_window`: 1024
Both `max_new_tokens` (=256) and `max_length`(=142) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.Image copyright PAWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.Image copyright PAThe Welsh Government said more people than ever were getting help to address housing problems.Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.Image copyright PAPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.However, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.Image copyright PAAndrew Stevens, who works in Welsh prison

In [10]:
# Display comparison table
comparison_df = pd.DataFrame(comparison_results)

print("\n📊 Model Comparison Table:")
print("=" * 80)
print(comparison_df.to_string(index=False))

print("\n💡 Key Insights:")
print("=" * 80)
print("BART-CNN: Longest, most informative (production choice)")
print("⚡ PEGASUS-XSum: Shortest, single sentence (headline style)")
print("LED-16K: Similar to BART, better for very long docs")


📊 Model Comparison Table:
       Model                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

## 7. Batch Processing

Demonstrates efficient processing of multiple documents.

In [11]:
# Batch processing with BART
print("📦 Batch Processing Demo\n")

# Get 10 documents
batch_docs = [samples[i]['document'] for i in range(10)]
batch_refs = [samples[i]['summary'] for i in range(10)]

print(f"Processing {len(batch_docs)} documents...\n")

start = time.time()
# Added truncation=True
batch_results = bart_summarizer(
    batch_docs,
    max_length=142,
    min_length=56,
    do_sample=False,
    batch_size=4,  # Process 4 at a time
    truncation=True
)
total_time = time.time() - start

batch_summaries = [r['summary_text'] for r in batch_results]

print("✅ Batch processing complete!\n")
print(f"Total time: {total_time:.2f}s")
print(f"Avg per document: {total_time/len(batch_docs):.2f}s")
print(f"Throughput: {len(batch_docs)/total_time:.2f} docs/second")

📦 Batch Processing Demo

Processing 10 documents...



NameError: name 'bart_summarizer' is not defined

In [ ]:
# Display batch results
if 'batch_summaries' in locals() and 'batch_refs' in locals() and 'batch_docs' in locals():
    batch_df = pd.DataFrame({
        'Article': [d[:100] + '...' for d in batch_docs[:5]],
        'BART Summary': [s[:100] + '...' for s in batch_summaries[:5]],
        'XSum Ref': batch_refs[:5],
        'BART Words': [len(s.split()) for s in batch_summaries[:5]],
        'Ref Words': [len(r.split()) for r in batch_refs[:5]]
    })

    print("\n📋 Sample Results (first 5):")
    print("=" * 80)
    print(batch_df.to_string(index=False))
else:
    print("⚠️ Batch results not available. Please run the Batch Processing cell successfully.")

## 8. Performance Analysis

In [12]:
# Calculate metrics
from statistics import mean

if 'batch_summaries' in locals() and batch_summaries:
    metrics = {
        'Average BART Length': mean([len(s.split()) for s in batch_summaries]),
        'Average XSum Length': mean([len(r.split()) for r in batch_refs]),
        'Compression Ratio': mean([
            len(batch_summaries[i].split()) / len(batch_docs[i].split()) * 100
            for i in range(len(batch_docs))
        ])
    }

    print("\n📊 Performance Metrics:")
    print("=" * 80)
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

    print("\n💡 Analysis:")
    print("=" * 80)
    print(f"✅ BART produces {metrics['Average BART Length']/metrics['Average XSum Length']:.1f}x longer summaries than XSum")
    print("✅ More informative for business use")
    print("✅ Compresses original to ~{:.1f}% of original length".format(metrics['Compression Ratio']))
else:
    print("⚠️ Metrics cannot be calculated because batch summaries are missing.")

⚠️ Metrics cannot be calculated because batch summaries are missing.


## 9. Save Results

In [13]:
# Save comprehensive results
if 'batch_summaries' in locals() and batch_summaries:
    results_df = pd.DataFrame({
        'Document': batch_docs,
        'BART_Summary': batch_summaries,
        'XSum_Reference': batch_refs,
        'BART_Words': [len(s.split()) for s in batch_summaries],
        'XSum_Words': [len(r.split()) for r in batch_refs]
    })

    results_df.to_csv('summarization_results.csv', index=False)
    print("✅ Results saved to 'summarization_results.csv'")
    print(f"   Total entries: {len(results_df)}")
else:
    print("⚠️ No results to save.")

⚠️ No results to save.
